### the gitclone command: git clone https://github.com/getian107/PRScs.git
### this part the package required is scipy and h5py, I used conda enviroment
### note update: Import random module from numpy instead of scipy

### ref_dir: LD_block info is using the 1kg_eur reference panel, downloaded from github page https://github.com/getian107/PRScs (can be update to UKB or other ethnic group)
### bim_prefix: the genotype file, I used UKB42krep.QC, and this file is used to provide a list of SNPs that are available in the target dataset
### sst_file: the summary statistics file, I used IQ.gwas, and this summary statistics file must include either BETA/OR + SE or BETA/OR + P, and it is strictly SNP/tA1/tA2/tBETA/tSE/n, etc.
### n_gwas: the sample size of the summary statistics file, I used 74214, and this number is used to calculate the effective sample size


In [ ]:
# the gitclone command: git clone https://github.com/getian107/PRScs.git
# this part the package required is scipy and h5py, I used conda enviroment
# note update: Import random module from numpy instead of scipy

# ref_dir: LD_block info is using the 1kg_eur reference panel, downloaded from github page https://github.com/getian107/PRScs (can be update to UKB or other ethnic group)
# bim_prefix: the genotype file, I used UKB42krep.QC, and this file is used to provide a list of SNPs that are available in the target dataset
# sst_file: the summary statistics file, I used IQ.gwas, and this summary statistics file must include either BETA/OR + SE or BETA/OR + P, and it is strictly SNP/tA1/tA2/tBETA/tSE/n, etc.
# n_gwas: the sample size of the summary statistics file, I used 74214, and this number is used to calculate the effective sample size

import subprocess

comm = f"python3 /PRScs-master/PRScs.py \
--ref_dir=LD_reference/ldblk_1kg_eur \
--bim_prefix=Genotype/UKB/UKB42krep.QC \
--sst_file=IQ.gwas \
--n_gwas=74214 \
--out_dir=PRS/temp/IQ"


subprocess.run(comm,check=True)

In [ ]:

input_dir = "PRS/temp/"
output_file = "IQ/PRS/Score/IQ.score"

with open(output_file, 'w') as f_out:
    for j in range(1, 23):  # from chr 1 to 22
        input_file = f"{input_dir}IQ_pst_eff_a1_b0.5_phiauto_chr{j}.txt"
        print(f"Appending file {input_file}")  
        try:
            with open(input_file, 'r') as f_in:
                f_out.write(f_in.read())  # 
        except FileNotFoundError:
            print(f"Warning: {input_file} not found. Skipping.")
        
print("All files have been successfully appended.")


### remove duplicate SNPs, keeping the first occurrence

In [ ]:
# Define input and output file paths
input_file = "IQ/PRS/Score/IQ.score"
output_file = "IQ/PRS/Score/IQ.cleaned.score"

# Initialize an empty set to track seen SNPs
seen_snp = set()

# Open the input file for reading and output file for writing
with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
        # Split the line into fields (assuming space or tab separated)
        fields = line.strip().split()
        
        # Check if the SNP (second column) has been seen before
        snp = fields[1]
        
        # If the SNP is not in the seen set, write the line to the output file
        if snp not in seen_snp:
            seen_snp.add(snp)
            outfile.write(line)

print(f"Duplicates removed. Cleaned data saved to {output_file}.")


### Plink for final output

In [ ]:
#Column 2: SNP IDs (the identifiers of the SNPs).
#Column 4: A1 (the allele for which the effect size is estimated).
#Column 6: The posterior effect size for each SNP (often the regression coefficient).

comm = f" /bin/plink1.9/plink \
--bfile /Genotype/UKB/UKB42krep.QC \
--score /IQ/PRS/Score/IQ.cleaned.score 2 4 6 \
--out /IQ/PRS/IQ"

subprocess.run(comm,check=True)